### Import du dataset

In [1]:
# Chargement package
import pandas as pd
import numpy as np

In [ ]:
# Connexion à ggdrive
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# Import du fichier via ggdrive
#file_path = "/content/drive/MyDrive/M2_SISE/Python_M2/df_clean.csv" # Lien perso ggdrive
df_nett = pd.read_csv("model_data.csv", sep = ',')

### Decision Tree

In [4]:
df_clean = df_nett.copy()
df_clean.head(2)

,iid_pid,race,date,career_c,age_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,age,imprace,clubbing,concerts,music,match
0,6804,4.0,6.0,1.0,29.0,3.0,10.0,10.0,3.0,10.0,0.0,27.0,4.0,3.0,6.0,5.0,0
1,6803,4.0,6.0,1.0,23.0,3.0,9.0,10.0,3.0,10.0,2.0,27.0,4.0,3.0,6.0,5.0,0


In [5]:
X = df_clean[df_clean.columns[1:-1]] # On enlève la colonne ID et la colonne CLASSE
y = df_clean.match # Colonne var à prédire

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6804 entries, 0 to 6803
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   race      6804 non-null   float64
 1   date      6804 non-null   float64
 2   career_c  6804 non-null   float64
 3   age_o     6804 non-null   float64
 4   attr_o    6804 non-null   float64
 5   sinc_o    6804 non-null   float64
 6   intel_o   6804 non-null   float64
 7   fun_o     6804 non-null   float64
 8   amb_o     6804 non-null   float64
 9   shar_o    6804 non-null   float64
 10  age       6804 non-null   float64
 11  imprace   6804 non-null   float64
 12  clubbing  6804 non-null   float64
 13  concerts  6804 non-null   float64
 14  music     6804 non-null   float64
dtypes: float64(15)
memory usage: 797.5 KB


In [7]:
print(X.shape, y.shape)

(6804, 15) (6804,)


In [8]:
y.value_counts()

0    5658
1    1146
Name: match, dtype: int64

In [191]:
# Recodage pour dummy sur var categorielles
ls_categ = ['race','date','career_c'] 

for i in ls_categ:
    X[i] = X[i].astype("category")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [9]:
# Utilisation du model_selection de sklearn
import sklearn
from sklearn import model_selection
# Affichage de la version
print(sklearn.__version__)

1.0.2


In [209]:
# Ajout d'une pour la modalité 16 absente de career dans df_clean mais présente dans submissions
X_dummy = pd.get_dummies(X)
X_dummy["career_c_16.0"] = 0

In [136]:
X_dummy.head(1)

,age_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,age,imprace,clubbing,...,career_c_9.0,career_c_10.0,career_c_11.0,career_c_12.0,career_c_13.0,career_c_14.0,career_c_15.0,career_c_17.0,career_c_99.0,career_c_16.0
0,29.0,3.0,10.0,10.0,3.0,10.0,0.0,27.0,4.0,3.0,...,0,0,0,0,0,0,0,0,0,0


In [170]:
X_dummy.shape

(6804, 44)

In [190]:
y = y.astype('int')
y

0       0
1       0
2       0
3       0
4       0
       ..
6799    1
6800    1
6801    1
6802    0
6803    0
Name: match, Length: 6804, dtype: int64

In [169]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_dummy,y, train_size=0.8, random_state=0, stratify=y)


In [141]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(5443, 44) (1361, 44) (5443,) (1361,)


In [142]:
# Vérification de l'égalité de distribution entre y_app et y_test
y_train.value_counts(normalize=True)

0    0.831527
1    0.168473
Name: match, dtype: float64

In [143]:
# Vérification de l'égalité de distribution entre y_app et y_test
y_test.value_counts(normalize=True)

0    0.831741
1    0.168259
Name: match, dtype: float64

In [23]:
# Load libraries
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score


In [ ]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(random_state=0)

# Train Decision Tree Classifer
model = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)


In [ ]:
clf.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 0,
 'splitter': 'best'}

In [ ]:
pd.DataFrame(model.feature_importances_, index = X_train.columns, columns = ['Importance']).sort_values(by=['Importance', ],ascending=False)

,Importance
fun_o,0.069771
age_o,0.051732
attr_o,0.047715
pf_o_int,0.041662
int_corr,0.040955
...,...
goal_6.0,0.000000
field_cd_6.0,0.000000
goal_4.0,0.000000
race_Missing value,0.000000


In [ ]:
precision_score(y_test,y_pred, pos_label = 1)

0.2701298701298701

In [ ]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.86      0.83      0.84      1698
           1       0.27      0.30      0.29       344

    accuracy                           0.74      2042
   macro avg       0.56      0.57      0.57      2042
weighted avg       0.76      0.74      0.75      2042



In [171]:
model2 = DecisionTreeClassifier(min_samples_split = 30, min_samples_leaf = 20)

# Train Decision Tree Classifer
model2.fit(X_train,y_train)

DecisionTreeClassifier(min_samples_leaf=20, min_samples_split=30)

In [172]:
y_pred2 = model2.predict(X_test)

In [173]:
print(metrics.classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.85      0.94      0.89      1132
           1       0.39      0.19      0.26       229

    accuracy                           0.81      1361
   macro avg       0.62      0.57      0.58      1361
weighted avg       0.77      0.81      0.79      1361



### Import Submission et préparation des données

In [192]:
# Import des données
sub = pd.read_csv("sub_clean.csv", sep = ',', index_col = [0])
sub.head(2)

,iid_pid,gender,condtn,samerace,race_o,field_cd,race,goal,date,go_out,...,music,shopping,yoga,exphappy,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1
0,6805,0,2,1,4,6.0,4,2,4,2,...,7.0,7.0,7.0,6.0,10.0,20.0,30.0,10.0,10.0,20.0
1,6806,0,2,1,4,6.0,4,2,4,2,...,7.0,7.0,7.0,6.0,10.0,20.0,30.0,10.0,10.0,20.0


In [193]:
# Subset des données avec les variables conservées pour le modèle
sub_test = sub[df_clean.columns[:-1].tolist()]
print(sub_test.shape)
sub_test.head(2)

(1574, 16)


,iid_pid,race,date,career_c,age_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,age,imprace,clubbing,concerts,music
0,6805,4,4,16.0,25.0,7.0,10.0,10.0,10.0,10.0,8.0,28.0,2.0,4.0,5.0,7.0
1,6806,4,4,16.0,26.0,6.0,7.0,8.0,6.0,8.0,8.0,28.0,2.0,4.0,5.0,7.0


In [194]:
# Préparation Xsub sans l'id
Xsub = sub_test[sub_test.columns[1:]]
Xsub.head(1)

,race,date,career_c,age_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,age,imprace,clubbing,concerts,music
0,4,4,16.0,25.0,7.0,10.0,10.0,10.0,10.0,8.0,28.0,2.0,4.0,5.0,7.0


In [195]:
# Recodage pour dummy sur var categorielles
ls_categ = ['race','date','career_c'] 

for i in ls_categ:
    Xsub[i] = Xsub[i].astype("category")

# Une fonction aurait été possible

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [196]:
# Ajustement du format des variables catégorielles race et date
# (Pour avoir le même format et donc nom de colonne que le dummy sur df_clean)
Xsub.race = Xsub.race.map('{:.1f}'.format)
Xsub.date = Xsub.date.map('{:.1f}'.format)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [197]:
# Création du dummy sur submissions
Xsub_dummy = pd.get_dummies(Xsub)
Xsub_dummy.shape

(1574, 37)

In [198]:
# Différence de nombre de colonnes car certaines modalités sont absentes dans submissions
X_dummy.shape

(6804, 44)

In [203]:
# Ajout des colonnes vides manquantes pour Xsub_dummy
col_missing = list(set(X_dummy.columns.tolist()).difference(set(Xsub_dummy.columns.tolist())))
col_missing

['career_c_14.0',
 'career_c_17.0',
 'date_99.0',
 'career_c_8.0',
 'career_c_12.0',
 'race_99.0',
 'date_1.0']

In [204]:
for j in col_missing:
  Xsub_dummy[j] = 0

In [202]:
Xsub_dummy.shape

(1574, 37)

In [201]:
# Vérification des variables dans XSub mais pas dans X
list(set(Xsub_dummy.columns.tolist()).difference(set(X_dummy.columns.tolist())))

[]

In [205]:
# Re-arrangement des colonnes à la manière de X
Xsub_dummy = Xsub_dummy[X_dummy.columns.tolist()]

In [206]:
# Prédiction avec model2, DecisionTree avec paramètres
ysub_pred = model2.predict(Xsub_dummy)

In [176]:
target = pd.DataFrame(ysub_pred)
target.value_counts(normalize= True)

0    0.959339
1    0.040661
dtype: float64

Préparation des données à l'export sur kaggle

In [164]:
sub_kaggle = pd.concat([sub.iloc[:,0].reset_index(drop=True), target], axis=1)

In [165]:
sub_kaggle.columns = ['iid_pid','target']

In [166]:
sub_kaggle.shape

(1574, 2)

In [167]:
from google.colab import files
sub_kaggle.to_csv('sub_kaggle3.csv', index = False)
files.download('sub_kaggle3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### SMOTE - Decision Tree

In [34]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

In [179]:
sm = SMOTE(random_state=42, sampling_strategy=0.5, k_neighbors=5)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [180]:
# Create Decision Tree classifer object
clf2 = DecisionTreeClassifier( random_state=0,min_samples_split=15,min_samples_leaf=20)

# Train Decision Tree Classifer
clf2 = clf2.fit(X_res, y_res)

#Predict the response for test dataset
y_pred2 = clf2.predict(X_test)


In [181]:
print(metrics.classification_report(y_test,y_pred2))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87      1132
           1       0.32      0.25      0.28       229

    accuracy                           0.78      1361
   macro avg       0.59      0.57      0.58      1361
weighted avg       0.76      0.78      0.77      1361



In [208]:
# Prédiction avec clf2, DecisionTree avec paramètres et smote sur les données
ysub_pred = clf2.predict(Xsub_dummy)

In [184]:
target = pd.DataFrame(ysub_pred)
target.value_counts()

0    1339
1     235
dtype: int64

Préparation des données à l'export sur kaggle

In [185]:
sub_kaggle = pd.concat([sub.iloc[:,0].reset_index(drop=True), target], axis=1)

In [186]:
sub_kaggle.columns = ['iid_pid','target']

In [187]:
sub_kaggle.shape

(1574, 2)

In [188]:
sub_kaggle.target.value_counts()

0    1339
1     235
Name: target, dtype: int64

In [189]:
from google.colab import files
sub_kaggle.to_csv('sub_kaggle4.csv', index = False)
files.download('sub_kaggle4.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>